This notebook is about an algorithm that can find the n-th regular number in linear time O(n). According to [Wikipedia](https://en.wikipedia.org/wiki/Regular_number#Number_theory), a regular number is formally defined as a number of the form $2^i \cdot 3^j \cdot 5^k$ where $i,j$ and $k$ are nonnegative integers. What make this problem interesting is that a similar problem was asked during Google interview, as reported in [this post on Stackoverflow](https://stackoverflow.com/questions/5505894/tricky-google-interview-question/5506635).

[In another post on  Stackoverflow](https://stackoverflow.com/questions/4600048/n%e1%b5%97%ca%b0-ugly-number), many solutions to find the n-th regular number are proposed. One among them is my own solution written in Python (here is the [link to my solution](https://stackoverflow.com/a/9007029/718529)).

# Outline

From this point on, I will discuss my solution. In section [The Algorithm](#The-Algorithm), the code of my algorithm is given. Next, instead of writting a mathematical proof to show that my algorithm is correct, I will verify it by comparing
the algorithm's generated sequence to datasets of regular number: 
- In section [Test Against Wikipedia's Data](#Test-Against-Wikipedia's-Data), a small dataset of size 26 is drawn from Wikipedia page 
- In section [Test Against Synthetic Data](#Test-Against-Synthetic-Data), a dataset of regular numbers less than or equal to 1000 is generated by brute force method


# The Algorithm

Below code will produce a sequence of regular number in ascending order. Starting from the first iteration, the algorithm takes O(1) to ouput `num=1` via `yield num` and O(1) for appending 1\*2, 1\*3, and 1\*5 to the three FIFO queues `qs`. Likewise, in later iteration (in `while` loop), it takes O(1) to (1) retrieve the next regular number, (2) output `num` and (3) append `num*2, num*3, num*5` to `qs`. Because of this, the total runtime from iteration 1 to iteration n will be O(n). When we run through iteration 1 to iteration n, the algorithm will generate a sequence of the first, the second, and so on up to the n-th regular numbers.


More detailed explanation as well as some illustrations can be found in:
- The section [Breif Explanation About the Algorithm](#Breif-Explanation-About-the-Algorithm) and,
- The sub-section [Example](#Example).

In [1]:
from collections import deque
bases = [2,3,5]

def regNumGenerator():
    qs = [deque([]) for _ in bases]
    counter = 1
    num = 1
    
    yield num
    print("Iteration {}".format(counter))
    print("num={}\n".format(num))
    [q.append(num*b) for q, b in zip(qs, bases)]
    
    counter+= 1
    
    while True:
        
        print("Iteration {}".format(counter))
        print("Queues Before popping: {}".format(qs))
        
        # (1) Retrieve the next regular number `num` by popping the smallest number from heads of the three queues
        num = min([q[0] for q in qs])
        [q.popleft() for q in qs if q[0] == num]
        
        print("num={}".format(num))
        print("Queues After poping: {}\n".format(qs))
        
        # (2) output the current regular number `num`
        yield num
        # (3) Add new members to tails of the three queues
        [q.append(num*b) for q, b in zip(qs, bases)]
        
        counter += 1
    

# Test Against Wikipedia's Data

On [the Wikipedia page on regular number](https://en.wikipedia.org/wiki/Regular_number#Number_theory), a sequence of regular numbers less than and up to 60 is given.
> The first few regular numbers are <br>
1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60

In [2]:
wikidata = [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]

In [3]:
it = regNumGenerator()
my_generated_seq = []
for num in it:
    if num > 60:
        break
    else: my_generated_seq.append(num)

Iteration 1
num=1

Iteration 2
Queues Before popping: [deque([2]), deque([3]), deque([5])]
num=2
Queues After poping: [deque([]), deque([3]), deque([5])]

Iteration 3
Queues Before popping: [deque([4]), deque([3, 6]), deque([5, 10])]
num=3
Queues After poping: [deque([4]), deque([6]), deque([5, 10])]

Iteration 4
Queues Before popping: [deque([4, 6]), deque([6, 9]), deque([5, 10, 15])]
num=4
Queues After poping: [deque([6]), deque([6, 9]), deque([5, 10, 15])]

Iteration 5
Queues Before popping: [deque([6, 8]), deque([6, 9, 12]), deque([5, 10, 15, 20])]
num=5
Queues After poping: [deque([6, 8]), deque([6, 9, 12]), deque([10, 15, 20])]

Iteration 6
Queues Before popping: [deque([6, 8, 10]), deque([6, 9, 12, 15]), deque([10, 15, 20, 25])]
num=6
Queues After poping: [deque([8, 10]), deque([9, 12, 15]), deque([10, 15, 20, 25])]

Iteration 7
Queues Before popping: [deque([8, 10, 12]), deque([9, 12, 15, 18]), deque([10, 15, 20, 25, 30])]
num=8
Queues After poping: [deque([10, 12]), deque([9, 

In [4]:
#Verify by comparing against data from Wiki
print(my_generated_seq)
print(wikidata)

assert my_generated_seq == wikidata

[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]
[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]


# Breif Explanation About the Algorithm
```text
Iteration 1:
    output 1
    add new member (2, 3, 5) to the queue structure
    
Iteration 2:
    Pop minimum number from the queue structure which is 2
    output 2
    add new members (2*2, 2*3 and 2*5) to the queue structure
                            ...

Iteration i-th:
    Pop minimum number from the queue structure. Calling it `num`
    output `num`
    add new members (num*2, num*3 and num*5) to the queue structure
```

The queue structure consists of three sub queues.

In each iteration of the algorithm, there are two stages:
1. We know that the first regular number is 1. Suppose that the algorithm know that the i-th regular number is `num`. Then, `num` will be an output of the algorithm in iteration i. Near the end of the i-th iteration, the queue structure will be updated . This is done by adding `num*2` to the tail of the first sub queue, adding `num*3` the the tail of the second sub queue, and adding `num*5` to the tail of the third sub queue. 
2. From iteration 2 onwards, the queue structure will be non-empty when we enter the iteration. So, at the beginning of the iteration i (i$\geq$2), we can retrieve the i-th regular number from the queue structure. The algorithm does this by popping the least number from heads of the three sub queue. This newly discovered number will be called `num`, and it is used in stage 1.


## Example

Let have a look at the state of the queue structure inside iteration 9 and 10 (the following text is taken from the long output in previous section) :
```
Iteration 9
Queues Before popping: [deque([10, 12, 16, 18]), deque([12, 15, 18, 24, 27]), deque([10, 15, 20, 25, 30, 40, 45])]
num=10
Queues After poping: [deque([12, 16, 18]), deque([12, 15, 18, 24, 27]), deque([15, 20, 25, 30, 40, 45])]

Iteration 10
Queues Before popping: [deque([12, 16, 18, 20]), deque([12, 15, 18, 24, 27, 30]), deque([15, 20, 25, 30, 40, 45, 50])]
num=12
```
Inside iteration 9, num=10 is retrieved from heads of the three sub queues.
![Popping ten](regularNum01.png)

Afterwards, the algorithm output num=10 as the 9-th regular number.<br> 
Next, before the transition from iteration 9 to iteration 10 occurs, the algorithm adds three numbers to tails of three sub queus: 20 (10\*2), 30 (10\*3) and 50 (10\*5).
![Adding ten times bases](regularNum02.png)

# Test Against Synthetic Data



In [26]:
#The algorithm will keep generating regular numbers until the number exceeds cutoff=1000.
cutoff = 1000

#def exponentFinder(cutoff):
#    def finder(base):
#        exponent = 0
#        while(base**exponent <= cutoff):
#            exponent += 1
#        return exponent
#    return finder

class exponentFinder:
    
    def __init__(self, cutoff):
        self.cutoff = cutoff
        
    def __call__(self, base):
        e = 0
        while base**e <= cutoff:
            e += 1
        return e

In [30]:
finder = exponentFinder(1000)

base = 5
exponent = finder(base)
print(exponent)
print(base**(exponent-1))
print(base**exponent)

5
625
3125


In [24]:
finder = exponentFinder2(1000)

base = 3
exponent = finder(base)
print(exponent)
print(base**(exponent-1))
print(base**exponent)

7
729
2187


In [5]:
from itertools import product

for x in product([1,2,8,16],[1,3,9,27], [1,5,25,125]):
    print(x)

(1, 1, 1)
(1, 1, 5)
(1, 1, 25)
(1, 1, 125)
(1, 3, 1)
(1, 3, 5)
(1, 3, 25)
(1, 3, 125)
(1, 9, 1)
(1, 9, 5)
(1, 9, 25)
(1, 9, 125)
(1, 27, 1)
(1, 27, 5)
(1, 27, 25)
(1, 27, 125)
(2, 1, 1)
(2, 1, 5)
(2, 1, 25)
(2, 1, 125)
(2, 3, 1)
(2, 3, 5)
(2, 3, 25)
(2, 3, 125)
(2, 9, 1)
(2, 9, 5)
(2, 9, 25)
(2, 9, 125)
(2, 27, 1)
(2, 27, 5)
(2, 27, 25)
(2, 27, 125)
(8, 1, 1)
(8, 1, 5)
(8, 1, 25)
(8, 1, 125)
(8, 3, 1)
(8, 3, 5)
(8, 3, 25)
(8, 3, 125)
(8, 9, 1)
(8, 9, 5)
(8, 9, 25)
(8, 9, 125)
(8, 27, 1)
(8, 27, 5)
(8, 27, 25)
(8, 27, 125)
(16, 1, 1)
(16, 1, 5)
(16, 1, 25)
(16, 1, 125)
(16, 3, 1)
(16, 3, 5)
(16, 3, 25)
(16, 3, 125)
(16, 9, 1)
(16, 9, 5)
(16, 9, 25)
(16, 9, 125)
(16, 27, 1)
(16, 27, 5)
(16, 27, 25)
(16, 27, 125)


In [6]:
class foo:
    def __init__(self, x):
        self.x=x

    def __call__(self, y):
        self.x = self.x + 1
        return (self.x, y)

In [9]:
functor = foo(5)
x = 44354234
print('(1.1) ', functor(7))
print('(1.2) ', functor(8))
print('(1.2) ', functor(9))

(1.1)  (6, 7)
(1.2)  (7, 8)
(1.2)  (8, 9)


In [10]:
def bar(x):
    def f(y):
        nonlocal x
        x = x + 1
        return (x,y)
    return f

functor2 = bar(5)
x = 345234234
print('(2.1) ', functor2(7))
print('(2.2) ', functor2(8))
print('(2.3) ', functor2(9))

(2.1)  (6, 7)
(2.2)  (7, 8)
(2.3)  (8, 9)
